## There are 10 categories in Managertoday

焦點 https://www.nownews.com/cat/column/

要聞 https://www.nownews.com/cat/news-summary/

即時 https://www.nownews.com/cat/breaking/

娛樂 https://www.nownews.com/cat/entertainment/

新奇 https://www.nownews.com/cat/novelty/

生活 https://www.nownews.com/cat/life/

財經 https://www.nownews.com/cat/finance/

專題 https://www.nownews.com/cat/st/

全球 https://www.nownews.com/cat/news-global/

運動 https://www.nownews.com/cat/sport/


# Crawl news for 10 categories

In [1]:
news_links = ['column/', 'news-summary/', 'breaking/', 'entertainment/', 'novelty/', 'life/', 'finance/', 'st/', 'news-global/', 'sport/']
news_categories = ['焦點', '要聞', '即時', '娛樂', '新奇', '生活', '財經', '專題', '全球', '運動']
base_url = 'https://www.nownews.com/cat/'

In [2]:
import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime, timedelta

In [3]:
links = []
titeles = []
dates = []
contents = []
categories = []
item_id = []
photo_links = []

for i, url_short_name in enumerate(news_links):  

    category = news_categories[i] 

    category_url = base_url + url_short_name
    print("Getting categorical news:", category)

    req = requests.get(category_url)
    page = BeautifulSoup(req.text, 'lxml')

    items = page.find('div', {'class': re.compile("listBlk horizontal\w*")}).findAll('li')

    serial_no = 1
    for j, item in enumerate (items):
        title = item.find('div', {'class': "txt"}).text
        print(serial_no, '--', title )

        link = item.find('a').get('href')

        dt = item.find('p', {'class':"time"}).text

        try:
            if item.find('img').has_attr('data-src'):
                photo_link = item.find('img').get('data-src')
            else:
                photo_link = item.find('img').get('src')
     
        except:
            photo_link = ''

        categories.append(category)
        dates.append(dt)
        titeles.append(title)
        links.append(link)
        photo_links.append(photo_link)
        
        # item id
        t = datetime.strptime(dt, '%Y-%m-%d %H:%M')
        tstr = t.strftime("%Y%m%d")

        item_id.append(url_short_name + "_" + tstr + "_" + str(serial_no))
        serial_no += 1

        
        # There are some words we don't like to analyze, They should be removed.
        page = BeautifulSoup(requests.get( link ).text,'lxml')

        # Find content
        content = page.find('article',{'itemprop':"articleBody"})

        # 額外插入的圖片與文字說明區塊-->移除之
        try:
            for media in content.findAll('div', {'id':"viewdeo_dable"}):
                media.decompose()
        except:
            pass

        try:
            for promote in content.findAll('div', {'class':"ad-blk1"}):
                promote.decompose()
        except:
            pass

        try:
            for tag in content.findAll('span', {'class':"hash-tag"}):
                tag.decompose()
        except:
            pass

        # Remove unnecessary content
        cont = content.text
        cont = re.sub('（中央社記者.+?）','', cont)
        cont = re.sub('（編輯：.*','', cont) 
        cont = re.sub('（譯者：.*','', cont) 
        cont = re.sub('（中央社\w*）','', cont) 
        cont = re.sub('\n','', cont) 
        cont = re.sub('\r','', cont) 
        cont = re.sub('\t','', cont) 
        cont = re.sub(' ','', cont) 

        contents.append(cont)
        time.sleep(2)  # 遵守爬蟲禮節，請小睡一下

        if (j==20): # Here we crawl only 2 pieces of news for each category, in order to save time.
                break 


Getting categorical news: 焦點
1 -- 
政院擬推6部會組改草案　農委會升格農業部
（中央社記者賴于榛台北4日電）政院官員今天表示，經長期溝通協調，行政院預定推出新一波組織改造，共牽涉6個部會，包含農委會升格為農業部、環保署升格為環境部、內政部新設國家公園署，以及原子能委員會改制為核...
2022-05-04 22:53

2 -- 
北藝中心7月正式開幕　142場演出華麗亮相
（中央社台北4日電）緊鄰士林夜市的台北表演藝術中心7月將正式開幕，37檔節目、142場演出曝光，從開幕明華園戲劇總團打頭陣到融合馬戲與合唱團的「達文西的notebook」，要讓嘴刁的觀眾眼睛一亮。台北...
2022-05-04 22:39

3 -- 
宜蘭漁船走私「7億元安毒」！船長認罪供出上游獲減刑
（中央社記者沈如峰宜蘭縣4日電）宜蘭「珍旺2號」漁船去年走私總淨重651.8公斤安非他命毒品，船長陳鼎旺落網後，供詞讓檢方順利追查其他共犯，法院依法減刑，判決陳鼎旺有期徒刑8年，全案可上訴。檢方當時調...
2022-05-04 22:22

4 -- 
限五天！7-11七支冰棒100元　3款「買一送一」網讚：真香
隨著天氣逐漸回溫，超商通路開始端出冰品促銷，綜觀各家本周快閃優惠，最吸睛當屬7-11祭出情人果冰棒7支100元，換算下來一支冰棒約14元，對於現在冰品貴森森的超商冰櫃來說，算是頗親民的價格。全家則有森...
2022-05-04 21:58

5 -- 
快篩難買！蔡英文下令：不要只有實名制、要增加購買管道
實名制快篩上路至今引發許多民怨，兼任民進黨主席的總統蔡英文在今（4）日民進黨中常會嚴肅要求行政部門正視此事，並表示目前國內快篩試劑數量既然是夠的，那應該可以規劃另闢機制，讓民眾能更便利取得，不要只有實...
2022-05-04 21:52

Getting categorical news: 要聞
1 -- 
民調第一的李宇翔、林延鳳齊站台　力挺黃淑君過初選
民進黨新北市議員初選明（5）日最後一天，爭取板橋區議員提名的黃淑君，今（4）日做最後的衝刺；初選民調第一的新北市議員參選人李宇翔、台北市議員參選人林延鳳，也連袂化身「超級助選員」，盼將這股氣勢給一樣是...
2022-05-04 21:01

2 -- 
快篩難買民怨起　蔡英文要求讓民眾

## Save data

In [4]:
data = zip(item_id, dates, categories, title, contents, links, photo_links)
df = pd.DataFrame(list(data), columns=['item_id','date','category','title','content','link','photo_link'])

In [5]:
df.head(2)

,item_id,date,category,title,content,link,photo_link
0,column/_20220504_1,2022-05-04 22:53,焦點,\n,政院官員今天表示，經長期溝通協調，行政院預定推出新一波組織改造，共牽涉6個部會，包含農委會升...,https://www.nownews.com/news/5794858,https://media.nownews.com/nn_media/thumbnail/2...
1,column/_20220504_2,2022-05-04 22:39,焦點,中,緊鄰士林夜市的台北表演藝術中心7月將正式開幕，37檔節目、142場演出曝光，從開幕明華園戲劇...,https://www.nownews.com/news/5794973,https://media.nownews.com/nn_media/thumbnail/2...


In [6]:
df.shape

(50, 7)

In [7]:
df.content[0]

'政院官員今天表示，經長期溝通協調，行政院預定推出新一波組織改造，共牽涉6個部會，包含農委會升格為農業部、環保署升格為環境部、內政部新設國家公園署，以及原子能委員會改制為核能安全委員會。行政院組織法修正案於2010年經立法院三讀通過，明訂行政院轄下37個部會整併為29個，並自2012年啟動組織改造。行政院繼去年提出設立「數位發展部」、科技部改為「國家科學及技術委員會」及國防部增設「後備動員署」等組改草案，經立法院三讀通過後，行政院會明天擬通過行政院人事行政總處擬具的「行政院組織法」第3條修正草案暨內政部等機關所擬「內政部組織法」修正草案等49項組織法草案。政院官員晚間對中央社記者表示，明天將推出最新一波組織改造，一共牽涉6個部會，包含內政部、經濟部、交通部、農委會、環保署與原能會。官員說，改組後，部分部會名稱與任務，以及底下的三級機關有一些新增或是調整，例如農委會升格為農業部、環保署升格為環境部、內政部新設國家公園署，以及原能會改制為核能安全委員會。官員表示，原能會改制為核能安全委員會是從部會改制為獨立機關，性質類似運安會。官員指出，組改啟動至今前後10年餘，陸續推出相關組改，這波組改是因部會之間協調已經完成，大家都有共識，因此由人事總處提出相關草案。'

In [8]:
df.to_csv("now_category_news.csv", sep="|", index=False)

# News Preprocessing Using Ckiplab NLP Package

In [9]:
import pandas as pd
import numpy
from collections import Counter
from snownlp import SnowNLP
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker

df = pd.read_csv('now_category_news.csv', sep='|')

ws = CkipWordSegmenter(level=2)
pos = CkipPosTagger(level=2)
ner = CkipNerChunker(level=2)

tokens = ws(df.content)

tokens_pos = pos(tokens)

word_pos_pair = [list(zip(w, p)) for w, p in zip(tokens, tokens_pos)]

entity_list = ner(df.content)

allowPOS = ['Na', 'Nb', 'Nc', 'VA', 'VAC', 'VB', 'VC']

tokens_v2 = []
for wp in word_pos_pair:
    tokens_v2.append([w for w, p in wp if (len(w) >= 2) and p in allowPOS])

df['tokens'] = tokens
df['tokens_v2'] = tokens_v2
df['entities'] = entity_list
df['token_pos'] = word_pos_pair

allowPOS = ['Na', 'Nb', 'Nc', 'VC']

def word_frequency(wp_pair):
    filtered_words = []
    for word, pos in wp_pair:
        if (pos in allowPOS) & (len(word) >= 2):
            filtered_words.append(word)

    counter = Counter(filtered_words)
    return counter.most_common(200)


keyfreqs = []
for wp in word_pos_pair:
    topwords = word_frequency(wp)
    keyfreqs.append(topwords)

df['top_key_freq'] = keyfreqs

summary = []
sentiment = []
for text in df.content:
    sn = SnowNLP(text)
    summary.append(sn.summary())
    sentiment.append(round(sn.sentiments, 2))

df['summary'] = summary
df['sentiment'] = sentiment

df = df[[
    'item_id', 'date','category', 'title', 'content', 'sentiment', 'summary',
    'top_key_freq', 'tokens', 'tokens_v2', 'entities', 'token_pos', 'link',
    'photo_link'
]]

df.to_csv('now_news_preprocessed.csv', sep='|', index=False)

print("process OK!")

Inference: 100%|██████████| 1/1 [00:45<00:00, 45.90s/it]


process OK!


In [10]:
df = pd.read_csv('now_news_preprocessed.csv', sep='|')
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,column/_20220504_1,2022-05-04 22:53,焦點,\n,政院官員今天表示，經長期溝通協調，行政院預定推出新一波組織改造，共牽涉6個部會，包含農委會升...,0.0,"['原能會改制為核能安全委員會是從部會改制為獨立機關', '行政院繼去年提出設立「數位發展部...","[('行政院', 6), ('部會', 6), ('官員', 5), ('內政部', 5),...","['政院', '官員', '今天', '表示', '，', '經', '長期', '溝通',...","['政院', '官員', '長期', '溝通', '協調', '行政院', '推出', '組...","[NerToken(word='今天', ner='DATE', idx=(4, 6)), ...","[('政院', 'Nc'), ('官員', 'Na'), ('今天', 'Nd'), ('表...",https://www.nownews.com/news/5794858,https://media.nownews.com/nn_media/thumbnail/2...


## Count top 200 word frequency for each category

In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('now_news_preprocessed.csv',sep='|')

In [13]:
from collections import Counter

In [14]:
news_links = ['column/', 'news-summary/', 'breaking/', 'entertainment/', 'novelty/', 'life/', 'finance/', 'st/', 'news-global/', 'sport/']
news_categories = ['焦點', '要聞', '即時', '娛樂', '新奇', '生活', '財經', '專題', '全球', '運動']

In [15]:
allowedPOS=['Na', 'Nb', 'Nc']

In [16]:
def get_top_words():
    top_cate_words={}
    counter_all = Counter()
    for category in news_categories:
        df_group = df[df.category == category]
        words_group = []
        for row in df_group.token_pos:
            filtered_words = []
            for (word, pos) in eval(row):
                if (len(word) >= 2) & (pos in allowedPOS):
                    filtered_words.append(word)                
            words_group += filtered_words
        counter = Counter( words_group )
        counter_all += counter
        topwords = counter.most_common(200)
        top_cate_words[category] = topwords
    top_cate_words['全部'] = counter_all.most_common(200)
    return list(top_cate_words.items())

In [17]:
result = get_top_words()

In [18]:
top_group_words = get_top_words()

In [19]:
df_top_group_words = pd.DataFrame(top_group_words, columns = ['category','top_keys'])

In [20]:
df_top_group_words

,category,top_keys
0,焦點,"[(中心, 14), (北藝, 11), (毒品, 9), (民眾, 9), (實名制, 9..."
1,要聞,"[(黃淑君, 11), (初選, 10), (蔡英文, 8), (司法, 7), (議員, ..."
2,即時,"[(中心, 13), (台北, 11), (北藝, 11), (性平, 10), (毒品, ..."
3,娛樂,"[(胡宇威, 13), (王宇婕, 10), (阿翰, 9), (導演, 8), (臉書, ..."
4,新奇,"[(網友, 19), (高職, 14), (老鼠, 12), (超商, 9), (阿翰, 9..."
5,生活,"[(高雄, 12), (榮總, 10), (毒品, 9), (會員, 9), (好市多, 9..."
6,財經,"[(指數, 18), (產業, 14), (市場, 13), (中國, 13), (製造業,..."
7,專題,"[(台灣, 32), (女優, 26), (A片, 26), (產業, 23), (經紀人,..."
8,全球,"[(香港, 16), (新聞, 15), (人權, 12), (蘋果, 12), (馬斯克,..."
9,運動,"[(球員, 19), (主場, 13), (聯盟, 11), (例行賽, 11), (太陽,..."


In [21]:
df_top_group_words.to_csv('now_news_topkeys_with_category_via_token_pos.csv', index=False)

## Count top 200 persons via NER

In [22]:
df = pd.read_csv('now_news_preprocessed.csv',sep='|')

In [23]:
df.head(1)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,column/_20220504_1,2022-05-04 22:53,焦點,\n,政院官員今天表示，經長期溝通協調，行政院預定推出新一波組織改造，共牽涉6個部會，包含農委會升...,0.0,"['原能會改制為核能安全委員會是從部會改制為獨立機關', '行政院繼去年提出設立「數位發展部...","[('行政院', 6), ('部會', 6), ('官員', 5), ('內政部', 5),...","['政院', '官員', '今天', '表示', '，', '經', '長期', '溝通',...","['政院', '官員', '長期', '溝通', '協調', '行政院', '推出', '組...","[NerToken(word='今天', ner='DATE', idx=(4, 6)), ...","[('政院', 'Nc'), ('官員', 'Na'), ('今天', 'Nd'), ('表...",https://www.nownews.com/news/5794858,https://media.nownews.com/nn_media/thumbnail/2...


In [24]:
df.entities[0]

"[NerToken(word='今天', ner='DATE', idx=(4, 6)), NerToken(word='行政院', ner='ORG', idx=(17, 20)), NerToken(word='6', ner='CARDINAL', idx=(35, 36)), NerToken(word='農委會', ner='ORG', idx=(42, 45)), NerToken(word='農業部', ner='ORG', idx=(48, 51)), NerToken(word='環保署', ner='ORG', idx=(52, 55)), NerToken(word='內政部', ner='ORG', idx=(62, 65)), NerToken(word='國家公園署', ner='ORG', idx=(67, 72)), NerToken(word='行政院', ner='ORG', idx=(92, 95)), NerToken(word='2010年', ner='DATE', idx=(102, 107)), NerToken(word='立法院', ner='ORG', idx=(108, 111)), NerToken(word='行政院', ner='ORG', idx=(118, 121)), NerToken(word='37', ner='CARDINAL', idx=(123, 125)), NerToken(word='29', ner='CARDINAL', idx=(131, 133)), NerToken(word='2012年', ner='DATE', idx=(137, 142)), NerToken(word='行政院', ner='ORG', idx=(149, 152)), NerToken(word='去年', ner='DATE', idx=(153, 155)), NerToken(word='科技部', ner='ORG', idx=(167, 170)), NerToken(word='國家科學及技術委員會', ner='ORG', idx=(173, 183)), NerToken(word='國防部', ner='ORG', idx=(185, 188)), NerToken(wor

In [25]:
def NerToken(word, ner, idx):
    return ner,word


In [26]:
for ner,key in eval(df.entities[0]):
    print(ner,key)

DATE 今天
ORG 行政院
CARDINAL 6
ORG 農委會
ORG 農業部
ORG 環保署
ORG 內政部
ORG 國家公園署
ORG 行政院
DATE 2010年
ORG 立法院
ORG 行政院
CARDINAL 37
CARDINAL 29
DATE 2012年
ORG 行政院
DATE 去年
ORG 科技部
ORG 國家科學及技術委員會
ORG 國防部
ORG 立法院
ORG 行政院
DATE 明天
ORG 行政院
LAW 行政院組織法
CARDINAL 4
CARDINAL 9
ORG 政院
ORG 中央社
DATE 明天
CARDINAL 6
ORG 內政部
ORG 經濟部
ORG 交通部
ORG 農委會
ORG 環保署
ORG 原能會
CARDINAL 三
ORG 農委會
ORG 農業部
ORG 環保署
ORG 環境部
ORG 內政部
ORG 人事總處


In [27]:
from collections import Counter

In [28]:
allowedNE = ['PERSON']

news_categories = ['焦點', '要聞', '即時', '娛樂', '新奇', '生活', '財經', '專題', '全球', '運動']

In [29]:
def ne_word_frequency( a_news_ne ):
    filtered_words =[]
    for ner,word in a_news_ne:
        if (len(word) >= 2) & (ner in allowedNE):
            filtered_words.append(word)
    counter = Counter( filtered_words )
    return counter.most_common( 200 )

In [30]:
def get_top_ner_words():
    top_cate_ner_words={}
    words_all=[]
    for category in news_categories:
        df_group = df[df.category == category]
        words_group = []

        # concatenate terms in a category
        for row in df_group.entities:
            words_group += eval(row)

        # concatenate all terms
        words_all += words_group

        # Get top words by calling ne_word_frequency() function
        topwords = ne_word_frequency( words_group )
        top_cate_ner_words[category] = topwords

    topwords_all = ne_word_frequency(words_all)
    top_cate_ner_words['全部'] = topwords_all
    
    return list(top_cate_ner_words.items())
    # return top_cate_ne_words

In [31]:
%%time
hotPersons = get_top_ner_words()

CPU times: total: 15.6 ms
Wall time: 19.9 ms


In [32]:
hotPersons

[('焦點',
  [('陳鼎旺', 6),
   ('蔡英文', 5),
   ('王孟超', 3),
   ('呂宗平', 3),
   ('羅秉成', 3),
   ('劉若瑀', 1),
   ('魏海敏', 1),
   ('蔡明亮', 1),
   ('桑布伊', 1),
   ('林桂如', 1),
   ('陳彥斌', 1),
   ('董怡芬', 1),
   ('王嘉明', 1),
   ('唐美雲', 1),
   ('陳武康', 1),
   ('謝春德', 1),
   ('何志偉', 1)]),
 ('要聞',
  [('黃淑君', 11),
   ('蔡英文', 8),
   ('魯明哲', 6),
   ('翁茂鍾', 4),
   ('李宇翔', 3),
   ('何博文', 3),
   ('林延鳳', 2),
   ('謝子涵', 2),
   ('林靜儀', 2),
   ('林奇福', 2),
   ('顏南全', 2),
   ('蘇義洲', 2),
   ('柯文哲', 2),
   ('陳乃瑜', 1),
   ('卓冠廷', 1),
   ('蔡適應', 1)]),
 ('即時',
  [('吳思瑤', 7),
   ('陳鼎旺', 6),
   ('王孟超', 3),
   ('呂宗平', 3),
   ('劉若瑀', 1),
   ('魏海敏', 1),
   ('蔡明亮', 1),
   ('桑布伊', 1),
   ('林桂如', 1),
   ('陳彥斌', 1),
   ('董怡芬', 1),
   ('王嘉明', 1),
   ('唐美雲', 1),
   ('陳武康', 1),
   ('謝春德', 1),
   ('潘文忠', 1)]),
 ('娛樂',
  [('胡宇威', 13),
   ('王宇婕', 10),
   ('阿翰', 9),
   ('白宇', 7),
   ('陶晶瑩', 6),
   ('曾沛慈', 6),
   ('江惠儀', 5),
   ('聶遠', 4),
   ('劉詩詩', 3),
   ('廖士涵', 3),
   ('吉那', 3),
   ('李燕', 2),
   ('楊淑喻', 2),
   ('廖麗芳', 1),
   ('陳坤', 1),
   ('

In [33]:
df_hotPersons = pd.DataFrame(hotPersons, columns = ['category','top_keys'])

In [34]:
df_hotPersons

,category,top_keys
0,焦點,"[(陳鼎旺, 6), (蔡英文, 5), (王孟超, 3), (呂宗平, 3), (羅秉成,..."
1,要聞,"[(黃淑君, 11), (蔡英文, 8), (魯明哲, 6), (翁茂鍾, 4), (李宇翔..."
2,即時,"[(吳思瑤, 7), (陳鼎旺, 6), (王孟超, 3), (呂宗平, 3), (劉若瑀,..."
3,娛樂,"[(胡宇威, 13), (王宇婕, 10), (阿翰, 9), (白宇, 7), (陶晶瑩,..."
4,新奇,"[(阿翰, 9), (陶晶瑩, 6), (廖麗芳, 1), (高三來, 1)]"
5,生活,"[(陳鼎旺, 6), (呂宗平, 3), (彭佳芸, 2), (潘炤穎, 1), (陳其邁,..."
6,財經,"[(陳馨蕙, 4), (JiteshUbrani, 1)]"
7,專題,"[(鄧佳華, 12), (Morgan, 10), (孟若羽, 3), (阿嘉, 3), (..."
8,全球,"[(馬斯克, 8), (尹錫悅, 7), (文在寅, 5), (陳小姐, 4), (杜耀明,..."
9,運動,"[(尼奧, 3), (Anderson, 2), (妮妮, 2), (汪蔚傑, 2), (陳..."


In [35]:
df_hotPersons.to_csv('now_news_top_person_by_category_via_ner.csv', sep=',', index=False)